In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [19]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N

def g_theta(sample_size, D):
    loc = np.random.random_sample((sample_size,D))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
train_y = torch.zeros([sample_size, N])




for i in range(sample_size):
    train_y[i] = Tensor(vf(train_x[i])) + torch.randn(Tensor(vf(train_x[i])).size()) * 0.0001
    

In [3]:
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) + torch.randn(Tensor(vf(x[i])).size()) * 0.0001
    return out

In [4]:
def stopping_criteria(tol_vector, f_target, lower_bound, upper_bound):
    lower_tol_vector = f_target - tol_vector
    upper_tol_vector = f_target + tol_vector
    SUCCESS = True
    FAILURE = False
    for i in range(f_target.shape[0]):
            if (lower_bound[i] < lower_tol_vector[i]) or  (upper_bound[i] > upper_tol_vector[i]):
                SUCCESS = False  
            if ((lower_bound[i] > upper_tol_vector[i]) or  (upper_bound[i] < lower_tol_vector[i])):
                FAILURE = True
    return SUCCESS, FAILURE

In [5]:

x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
#         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
        self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    


In [6]:
###hyperparameters optimization###
def hyper_opti(g_theta1, agg_data, training_iter):
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

    model = MultitaskGPModel(g_theta1, agg_data, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(),  lr=0.1)  # Includes GaussianLikelihood parameters
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(g_theta1)
        #output_ll = likelihood(output)

        #loss = -likelihood.get_mll(agg_data,output_ll)
        loss = -mll(output, agg_data)
        loss.backward()

        print('Iter %d/%d - Loss hyperparam: %.3f' % (i + 1, training_iter, loss.item()))
        optimizer.step()

    return model, likelihood


In [7]:
class param_opti(nn.Module):
    def __init__(self, sample):
        super(param_opti, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        self.g_theta2 = nn.Parameter(Tensor(sample))
        
    def forward(self):
        
        return (self.g_theta2)

In [8]:
def conduct_param_opti(x,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_iter):
    model.eval()
    likelihood.eval()
    _par = param_opti(loc_sample)
    
    optimizer = torch.optim.Adam(_par.parameters(), lr=0.007)
    
    for j in range(training_iter):
        optimizer.zero_grad()
        g_theta2 = _par.forward()

        loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
        loss1 = -1 * loss1
        loss1.backward(retain_graph=True)
        print('Iter %d/%d - Loss theta2: %.3f' % (j + 1, training_iter, loss1.item()))
        optimizer.step()
          
    return loss1, g_theta2, lower_bound, upper_bound

In [9]:
class design_opti(nn.Module):
    def __init__(self, _x):
        super(design_opti, self).__init__()
#         a = _x[0,0]
#         b = _x[0,1]
#         self.x_design_1 = nn.Parameter(a)
#         self.x_design_2 = nn.Parameter(b)
        self.x_design = nn.Parameter(_x)
    def forward(self):
#         x_design = torch.zeros(1,2)
#         x_design[0,0] = self.x_design_1
#         x_design[0,1] = self.x_design_2
        return (self.x_design)

In [10]:
def conduct_design_opti(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter, training_param_iter):
    design = design_opti(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    def closure():
        optimizer.zero_grad()
        loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter)
        loss2.backward(retain_graph=True)
        
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=1., history_size=100, max_iter=10, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter) ##takethis out
    print('Loss design: %.3f' % ( loss2))
    print(optimizer.state)
    return x_d, g_theta2_out, lower_bound, upper_bound
    
    

In [11]:
iter_hp = 50
iter_design = 20
iter_param = 50

f_target = Tensor(vf.tgt_vec)
f_target = f_target.reshape(f_target.shape[0],1)
tol_vector = 0.0005 * torch.ones(f_target.shape)


loc_size = 10
loc_sample = np.random.random_sample((loc_size,2))



g_theta1 = x_train

agg_data = y_train.flatten()


x0 = Tensor(np.array([0.5,0.7]))
x0 = x0.reshape(1,2)
x00 = x0
vec_x = x00
SUCCESS = False
FAILURE = False
iter = 0
tol  = 0.009
while(SUCCESS == False and iter < 50):
    
    
    print('START HYPERPARAMETERS optimization')
    model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp)
    
    print('END HYPERPARAMETERS optimization')
    print(iter)
    x0_new,g_theta2,lower_bound, upper_bound = conduct_design_opti(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design,iter_param)
    print(lower_bound)
    print(upper_bound)
    print(f_target-tol_vector)
    print(f_target+tol_vector)
    loc_sample = np.random.random_sample((loc_size,2))
    x0 = x0_new #Tensor(np.random.random_sample((1,2))) #x0_new
    vec_x = torch.cat([vec_x, x0_new])
    g_theta2_detach = g_theta2.detach()
    new_data = vfield_(g_theta2_detach)
    agg_data = torch.cat([agg_data, new_data.flatten()], 0)
    g_theta1= torch.cat([g_theta1, g_theta2], 0)

    SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)
    iter = iter + 1

print(x0_new)
    

START HYPERPARAMETERS optimization
Iter 1/50 - Loss hyperparam: 0.846
Iter 2/50 - Loss hyperparam: 0.803
Iter 3/50 - Loss hyperparam: 0.760
Iter 4/50 - Loss hyperparam: 0.717
Iter 5/50 - Loss hyperparam: 0.674
Iter 6/50 - Loss hyperparam: 0.631
Iter 7/50 - Loss hyperparam: 0.588
Iter 8/50 - Loss hyperparam: 0.544
Iter 9/50 - Loss hyperparam: 0.501
Iter 10/50 - Loss hyperparam: 0.457
Iter 11/50 - Loss hyperparam: 0.413
Iter 12/50 - Loss hyperparam: 0.370
Iter 13/50 - Loss hyperparam: 0.327
Iter 14/50 - Loss hyperparam: 0.283
Iter 15/50 - Loss hyperparam: 0.239
Iter 16/50 - Loss hyperparam: 0.195
Iter 17/50 - Loss hyperparam: 0.150
Iter 18/50 - Loss hyperparam: 0.104
Iter 19/50 - Loss hyperparam: 0.057
Iter 20/50 - Loss hyperparam: 0.010
Iter 21/50 - Loss hyperparam: -0.037
Iter 22/50 - Loss hyperparam: -0.084
Iter 23/50 - Loss hyperparam: -0.131
Iter 24/50 - Loss hyperparam: -0.179
Iter 25/50 - Loss hyperparam: -0.228
Iter 26/50 - Loss hyperparam: -0.277
Iter 27/50 - Loss hyperparam: -0

Iter 6/50 - Loss theta2: -5.809
Iter 7/50 - Loss theta2: -5.814
Iter 8/50 - Loss theta2: -5.818
Iter 9/50 - Loss theta2: -5.822
Iter 10/50 - Loss theta2: -5.825
Iter 11/50 - Loss theta2: -5.829
Iter 12/50 - Loss theta2: -5.832
Iter 13/50 - Loss theta2: -5.835
Iter 14/50 - Loss theta2: -5.838
Iter 15/50 - Loss theta2: -5.841
Iter 16/50 - Loss theta2: -5.843
Iter 17/50 - Loss theta2: -5.846
Iter 18/50 - Loss theta2: -5.848
Iter 19/50 - Loss theta2: -5.850
Iter 20/50 - Loss theta2: -5.852
Iter 21/50 - Loss theta2: -5.854
Iter 22/50 - Loss theta2: -5.856
Iter 23/50 - Loss theta2: -5.857
Iter 24/50 - Loss theta2: -5.858
Iter 25/50 - Loss theta2: -5.860
Iter 26/50 - Loss theta2: -5.861
Iter 27/50 - Loss theta2: -5.862
Iter 28/50 - Loss theta2: -5.863
Iter 29/50 - Loss theta2: -5.864
Iter 30/50 - Loss theta2: -5.865
Iter 31/50 - Loss theta2: -5.866
Iter 32/50 - Loss theta2: -5.867
Iter 33/50 - Loss theta2: -5.868
Iter 34/50 - Loss theta2: -5.869
Iter 35/50 - Loss theta2: -5.869
Iter 36/50 - L

Iter 19/50 - Loss theta2: -6.081
Iter 20/50 - Loss theta2: -6.082
Iter 21/50 - Loss theta2: -6.084
Iter 22/50 - Loss theta2: -6.086
Iter 23/50 - Loss theta2: -6.087
Iter 24/50 - Loss theta2: -6.088
Iter 25/50 - Loss theta2: -6.090
Iter 26/50 - Loss theta2: -6.091
Iter 27/50 - Loss theta2: -6.092
Iter 28/50 - Loss theta2: -6.093
Iter 29/50 - Loss theta2: -6.094
Iter 30/50 - Loss theta2: -6.095
Iter 31/50 - Loss theta2: -6.096
Iter 32/50 - Loss theta2: -6.097
Iter 33/50 - Loss theta2: -6.098
Iter 34/50 - Loss theta2: -6.098
Iter 35/50 - Loss theta2: -6.099
Iter 36/50 - Loss theta2: -6.099
Iter 37/50 - Loss theta2: -6.100
Iter 38/50 - Loss theta2: -6.100
Iter 39/50 - Loss theta2: -6.101
Iter 40/50 - Loss theta2: -6.101
Iter 41/50 - Loss theta2: -6.102
Iter 42/50 - Loss theta2: -6.102
Iter 43/50 - Loss theta2: -6.103
Iter 44/50 - Loss theta2: -6.103
Iter 45/50 - Loss theta2: -6.103
Iter 46/50 - Loss theta2: -6.103
Iter 47/50 - Loss theta2: -6.103
Iter 48/50 - Loss theta2: -6.104
Iter 49/50

Iter 27/50 - Loss theta2: -6.278
Iter 28/50 - Loss theta2: -6.278
Iter 29/50 - Loss theta2: -6.279
Iter 30/50 - Loss theta2: -6.280
Iter 31/50 - Loss theta2: -6.281
Iter 32/50 - Loss theta2: -6.282
Iter 33/50 - Loss theta2: -6.282
Iter 34/50 - Loss theta2: -6.283
Iter 35/50 - Loss theta2: -6.284
Iter 36/50 - Loss theta2: -6.284
Iter 37/50 - Loss theta2: -6.285
Iter 38/50 - Loss theta2: -6.285
Iter 39/50 - Loss theta2: -6.286
Iter 40/50 - Loss theta2: -6.286
Iter 41/50 - Loss theta2: -6.286
Iter 42/50 - Loss theta2: -6.287
Iter 43/50 - Loss theta2: -6.287
Iter 44/50 - Loss theta2: -6.287
Iter 45/50 - Loss theta2: -6.288
Iter 46/50 - Loss theta2: -6.288
Iter 47/50 - Loss theta2: -6.288
Iter 48/50 - Loss theta2: -6.288
Iter 49/50 - Loss theta2: -6.288
Iter 50/50 - Loss theta2: -6.289
Iter 1/50 - Loss theta2: -6.216
Iter 2/50 - Loss theta2: -6.220
Iter 3/50 - Loss theta2: -6.224
Iter 4/50 - Loss theta2: -6.228
Iter 5/50 - Loss theta2: -6.231
Iter 6/50 - Loss theta2: -6.235
Iter 7/50 - Loss

Iter 32/50 - Loss theta2: -7.088
Iter 33/50 - Loss theta2: -7.088
Iter 34/50 - Loss theta2: -7.088
Iter 35/50 - Loss theta2: -7.088
Iter 36/50 - Loss theta2: -7.089
Iter 37/50 - Loss theta2: -7.089
Iter 38/50 - Loss theta2: -7.089
Iter 39/50 - Loss theta2: -7.089
Iter 40/50 - Loss theta2: -7.090
Iter 41/50 - Loss theta2: -7.090
Iter 42/50 - Loss theta2: -7.090
Iter 43/50 - Loss theta2: -7.090
Iter 44/50 - Loss theta2: -7.090
Iter 45/50 - Loss theta2: -7.090
Iter 46/50 - Loss theta2: -7.090
Iter 47/50 - Loss theta2: -7.090
Iter 48/50 - Loss theta2: -7.090
Iter 49/50 - Loss theta2: -7.090
Iter 50/50 - Loss theta2: -7.090
Iter 1/50 - Loss theta2: -6.614
Iter 2/50 - Loss theta2: -6.620
Iter 3/50 - Loss theta2: -6.625
Iter 4/50 - Loss theta2: -6.630
Iter 5/50 - Loss theta2: -6.634
Iter 6/50 - Loss theta2: -6.638
Iter 7/50 - Loss theta2: -6.642
Iter 8/50 - Loss theta2: -6.645
Iter 9/50 - Loss theta2: -6.648
Iter 10/50 - Loss theta2: -6.651
Iter 11/50 - Loss theta2: -6.654
Iter 12/50 - Loss t

...

Iter 19/50 - Loss theta2: -10.889
Iter 20/50 - Loss theta2: -10.890
Iter 21/50 - Loss theta2: -10.889
Iter 22/50 - Loss theta2: -10.889
Iter 23/50 - Loss theta2: -10.891
Iter 24/50 - Loss theta2: -10.891
Iter 25/50 - Loss theta2: -10.890
Iter 26/50 - Loss theta2: -10.891
Iter 27/50 - Loss theta2: -10.892
Iter 28/50 - Loss theta2: -10.891
Iter 29/50 - Loss theta2: -10.892
Iter 30/50 - Loss theta2: -10.892
Iter 31/50 - Loss theta2: -10.893
Iter 32/50 - Loss theta2: -10.892
Iter 33/50 - Loss theta2: -10.894
Iter 34/50 - Loss theta2: -10.894
Iter 35/50 - Loss theta2: -10.894
Iter 36/50 - Loss theta2: -10.894
Iter 37/50 - Loss theta2: -10.895
Iter 38/50 - Loss theta2: -10.895
Iter 39/50 - Loss theta2: -10.895
Iter 40/50 - Loss theta2: -10.895
Iter 41/50 - Loss theta2: -10.896
Iter 42/50 - Loss theta2: -10.895
Iter 43/50 - Loss theta2: -10.896
Iter 44/50 - Loss theta2: -10.896
Iter 45/50 - Loss theta2: -10.897
Iter 46/50 - Loss theta2: -10.898
Iter 47/50 - Loss theta2: -10.897
Iter 48/50 - L

Iter 41/50 - Loss hyperparam: -1.103
Iter 42/50 - Loss hyperparam: -1.155
Iter 43/50 - Loss hyperparam: -1.206
Iter 44/50 - Loss hyperparam: -1.257
Iter 45/50 - Loss hyperparam: -1.308
Iter 46/50 - Loss hyperparam: -1.359
Iter 47/50 - Loss hyperparam: -1.410
Iter 48/50 - Loss hyperparam: -1.460
Iter 49/50 - Loss hyperparam: -1.511
Iter 50/50 - Loss hyperparam: -1.562
END HYPERPARAMETERS optimization
49
Iter 1/50 - Loss theta2: -10.858
Iter 2/50 - Loss theta2: -10.858
Iter 3/50 - Loss theta2: -10.859
Iter 4/50 - Loss theta2: -10.859
Iter 5/50 - Loss theta2: -10.859
Iter 6/50 - Loss theta2: -10.859
Iter 7/50 - Loss theta2: -10.860
Iter 8/50 - Loss theta2: -10.860
Iter 9/50 - Loss theta2: -10.861
Iter 10/50 - Loss theta2: -10.861
Iter 11/50 - Loss theta2: -10.861
Iter 12/50 - Loss theta2: -10.861
Iter 13/50 - Loss theta2: -10.862
Iter 14/50 - Loss theta2: -10.862
Iter 15/50 - Loss theta2: -10.863
Iter 16/50 - Loss theta2: -10.863
Iter 17/50 - Loss theta2: -10.864
Iter 18/50 - Loss theta2:

Iter 38/50 - Loss theta2: 0.274
Iter 39/50 - Loss theta2: 0.273
Iter 40/50 - Loss theta2: 0.273
Iter 41/50 - Loss theta2: 0.272
Iter 42/50 - Loss theta2: 0.273
Iter 43/50 - Loss theta2: 0.272
Iter 44/50 - Loss theta2: 0.271
Iter 45/50 - Loss theta2: 0.272
Iter 46/50 - Loss theta2: 0.271
Iter 47/50 - Loss theta2: 0.271
Iter 48/50 - Loss theta2: 0.271
Iter 49/50 - Loss theta2: 0.271
Iter 50/50 - Loss theta2: 0.271
Iter 1/50 - Loss theta2: -6.692
Iter 2/50 - Loss theta2: -6.693
Iter 3/50 - Loss theta2: -6.695
Iter 4/50 - Loss theta2: -6.697
Iter 5/50 - Loss theta2: -6.696
Iter 6/50 - Loss theta2: -6.699
Iter 7/50 - Loss theta2: -6.699
Iter 8/50 - Loss theta2: -6.700
Iter 9/50 - Loss theta2: -6.701
Iter 10/50 - Loss theta2: -6.703
Iter 11/50 - Loss theta2: -6.702
Iter 12/50 - Loss theta2: -6.704
Iter 13/50 - Loss theta2: -6.704
Iter 14/50 - Loss theta2: -6.706
Iter 15/50 - Loss theta2: -6.706
Iter 16/50 - Loss theta2: -6.706
Iter 17/50 - Loss theta2: -6.707
Iter 18/50 - Loss theta2: -6.707

Iter 36/50 - Loss theta2: -10.923
Iter 37/50 - Loss theta2: -10.922
Iter 38/50 - Loss theta2: -10.922
Iter 39/50 - Loss theta2: -10.923
Iter 40/50 - Loss theta2: -10.923
Iter 41/50 - Loss theta2: -10.923
Iter 42/50 - Loss theta2: -10.924
Iter 43/50 - Loss theta2: -10.924
Iter 44/50 - Loss theta2: -10.925
Iter 45/50 - Loss theta2: -10.925
Iter 46/50 - Loss theta2: -10.925
Iter 47/50 - Loss theta2: -10.925
Iter 48/50 - Loss theta2: -10.926
Iter 49/50 - Loss theta2: -10.925
Iter 50/50 - Loss theta2: -10.926
Iter 1/50 - Loss theta2: -10.928
Iter 2/50 - Loss theta2: -10.929
Iter 3/50 - Loss theta2: -10.929
Iter 4/50 - Loss theta2: -10.929
Iter 5/50 - Loss theta2: -10.929
Iter 6/50 - Loss theta2: -10.930
Iter 7/50 - Loss theta2: -10.930
Iter 8/50 - Loss theta2: -10.931
Iter 9/50 - Loss theta2: -10.931
Iter 10/50 - Loss theta2: -10.931
Iter 11/50 - Loss theta2: -10.932
Iter 12/50 - Loss theta2: -10.932
Iter 13/50 - Loss theta2: -10.932
Iter 14/50 - Loss theta2: -10.933
Iter 15/50 - Loss theta

Iter 29/50 - Loss theta2: -10.938
Iter 30/50 - Loss theta2: -10.939
Iter 31/50 - Loss theta2: -10.939
Iter 32/50 - Loss theta2: -10.940
Iter 33/50 - Loss theta2: -10.940
Iter 34/50 - Loss theta2: -10.940
Iter 35/50 - Loss theta2: -10.941
Iter 36/50 - Loss theta2: -10.940
Iter 37/50 - Loss theta2: -10.941
Iter 38/50 - Loss theta2: -10.942
Iter 39/50 - Loss theta2: -10.941
Iter 40/50 - Loss theta2: -10.942
Iter 41/50 - Loss theta2: -10.943
Iter 42/50 - Loss theta2: -10.943
Iter 43/50 - Loss theta2: -10.943
Iter 44/50 - Loss theta2: -10.944
Iter 45/50 - Loss theta2: -10.944
Iter 46/50 - Loss theta2: -10.944
Iter 47/50 - Loss theta2: -10.945
Iter 48/50 - Loss theta2: -10.944
Iter 49/50 - Loss theta2: -10.945
Iter 50/50 - Loss theta2: -10.945
Iter 1/50 - Loss theta2: -10.928
Iter 2/50 - Loss theta2: -10.929
Iter 3/50 - Loss theta2: -10.929
Iter 4/50 - Loss theta2: -10.929
Iter 5/50 - Loss theta2: -10.929
Iter 6/50 - Loss theta2: -10.930
Iter 7/50 - Loss theta2: -10.930
Iter 8/50 - Loss thet

In [ ]:
#x0 = Tensor(np.array([0.1937, 0.1257]))
#x0 = Tensor(np.array([0.1885, 0.1038]))

x0 = Tensor(np.array([0.1990, 0.0979]))
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

In [ ]:
g_theta1.shape

In [20]:
sample_size_TAD = 450
train_x = g_theta(sample_size_TAD, D)
train_y = torch.zeros([sample_size_TAD, N])




for i in range(sample_size_TAD):
    train_y[i] = Tensor(vf(train_x[i])) + torch.randn(Tensor(vf(train_x[i])).size()) * 0.001

In [21]:
def conduct_design_opti_random(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter):
    design = design_opti(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    def closure():
        optimizer.zero_grad()
        loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d,g_theta1, agg_data, model, likelihood)
        loss2 = -1. * loss2
        loss2.backward(retain_graph=True)
        
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=1., history_size=100, max_iter=10, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d,g_theta1, agg_data, model, likelihood) ##takethis out
    loss2 = -1. * loss2
    print('Loss design: %.3f' % ( loss2))
    print(optimizer.state)
    return x_d, lower_bound, upper_bound

In [22]:
iter_hp = 50
iter_design = 50
iter_param = 50

f_target = Tensor(vf.tgt_vec)
f_target = f_target.reshape(f_target.shape[0],1)
tol_vector = 0.005 * torch.ones(f_target.shape)


loc_size = 20
loc_sample = np.random.random_sample((loc_size,2))



g_theta1 = train_x

agg_data = train_y.flatten()


x00 = Tensor(np.array([0.3,.2]))
x00 = x0.reshape(1,2)

SUCCESS = False
FAILURE = False

    
    
print('START HYPERPARAMETERS optimization')
model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp)
    
print('END HYPERPARAMETERS optimization')
x0_new,lower_bound, upper_bound = conduct_design_opti_random(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design)
print(x0_new)
print(lower_bound)
print(upper_bound)
print(f_target-tol_vector)
print(f_target+tol_vector)

START HYPERPARAMETERS optimization
Iter 1/50 - Loss hyperparam: 0.834
Iter 2/50 - Loss hyperparam: 0.788
Iter 3/50 - Loss hyperparam: 0.744
Iter 4/50 - Loss hyperparam: 0.700
Iter 5/50 - Loss hyperparam: 0.657
Iter 6/50 - Loss hyperparam: 0.614
Iter 7/50 - Loss hyperparam: 0.572
Iter 8/50 - Loss hyperparam: 0.529
Iter 9/50 - Loss hyperparam: 0.486
Iter 10/50 - Loss hyperparam: 0.443
Iter 11/50 - Loss hyperparam: 0.399
Iter 12/50 - Loss hyperparam: 0.355
Iter 13/50 - Loss hyperparam: 0.311
Iter 14/50 - Loss hyperparam: 0.266
Iter 15/50 - Loss hyperparam: 0.220
Iter 16/50 - Loss hyperparam: 0.175
Iter 17/50 - Loss hyperparam: 0.128
Iter 18/50 - Loss hyperparam: 0.081
Iter 19/50 - Loss hyperparam: 0.034
Iter 20/50 - Loss hyperparam: -0.013
Iter 21/50 - Loss hyperparam: -0.061
Iter 22/50 - Loss hyperparam: -0.109
Iter 23/50 - Loss hyperparam: -0.158
Iter 24/50 - Loss hyperparam: -0.207
Iter 25/50 - Loss hyperparam: -0.256
Iter 26/50 - Loss hyperparam: -0.305
Iter 27/50 - Loss hyperparam: -

In [ ]:
x0 = Tensor(np.array([0.2061, 0.0813]))
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

In [12]:
print(g_theta1.shape)

torch.Size([600, 2])


In [18]:
min(g_theta1[:,0])

tensor(-0.4300, grad_fn=<UnbindBackward>)